# Breast Cancer Classification — Notebook Version

This notebook mirrors `app.py` but uses native Python output instead of Streamlit.
All six models are evaluated on the test split and compared side by side.

## Imports and Setup

In [ ]:
from __future__ import annotations

import json
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    matthews_corrcoef,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "model").exists() and (PROJECT_ROOT.parent / "model").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

MODEL_DIR = PROJECT_ROOT / "model"
DATA_DIR  = PROJECT_ROOT / "data"

print(f"MODEL_DIR : {MODEL_DIR}")
print(f"DATA_DIR  : {DATA_DIR}")

## Model Map and Helper Functions

In [ ]:
MODEL_FILE_MAP = {
    "Logistic Regression": "logistic_regression.pkl",
    "Decision Tree": "decision_tree.pkl",
    "kNN": "knn.pkl",
    "Naive Bayes": "naive_bayes.pkl",
    "Random Forest (Ensemble)": "random_forest_ensemble.pkl",
    "XGBoost (Ensemble)": "xgboost_ensemble.pkl",
}


def load_model(name):
    return joblib.load(MODEL_DIR / MODEL_FILE_MAP[name])


def evaluate(y_true, y_pred, y_prob):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_prob),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }

## Load Dataset Metadata

In [ ]:
with open(MODEL_DIR / "dataset_metadata.json") as f:
    meta = json.load(f)

print(f"Dataset   : {meta['dataset_name']}")
print(f"Samples   : {meta['instances']}")
print(f"Features  : {meta['features']}")
print(f"Classes   : {', '.join(meta['target_names'])}")

## Load Test Data

In [ ]:
eval_df = pd.read_csv(DATA_DIR / "test_data.csv")
feature_cols = [c for c in eval_df.columns if c != "target"]

X_eval = eval_df[feature_cols]
y_true = eval_df["target"]

print(f"Test samples: {len(eval_df)}")
print(f"Features    : {len(feature_cols)}")
eval_df.head()

## Run All 6 Models

In [ ]:
all_metrics = []
model_predictions = {}

for name in MODEL_FILE_MAP:
    model = load_model(name)
    preds = model.predict(X_eval)
    probs = model.predict_proba(X_eval)[:, 1]
    model_predictions[name] = {"preds": preds, "probs": probs}

    metrics = evaluate(y_true, preds, probs)
    metrics["Model"] = name
    all_metrics.append(metrics)

print(f"Evaluated {len(all_metrics)} models.")

## All-Model Comparison Table

In [ ]:
metrics_df = pd.DataFrame(all_metrics)
metrics_df = metrics_df[["Model", "Accuracy", "AUC", "Precision", "Recall", "F1", "MCC"]]
metrics_df = metrics_df.sort_values("Accuracy", ascending=False).reset_index(drop=True)

metrics_df.style.format(
    {col: "{:.4f}" for col in ["Accuracy", "AUC", "Precision", "Recall", "F1", "MCC"]}
).background_gradient(cmap="Greens", subset=["Accuracy", "AUC", "F1"])

## Confusion Matrices (All Models)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.ravel()

for i, name in enumerate(MODEL_FILE_MAP):
    cm = confusion_matrix(y_true, model_predictions[name]["preds"])
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=axes[i])
    axes[i].set_title(name, fontsize=11)
    axes[i].set_xlabel("Predicted")
    axes[i].set_ylabel("Actual")

plt.tight_layout()
plt.show()

## ROC Curves (All Models)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

for name in MODEL_FILE_MAP:
    fpr, tpr, _ = roc_curve(y_true, model_predictions[name]["probs"])
    auc_val = roc_auc_score(y_true, model_predictions[name]["probs"])
    ax.plot(fpr, tpr, label=f"{name} (AUC={auc_val:.3f})")

ax.plot([0, 1], [0, 1], "k--", alpha=0.4)
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Comparison")
ax.legend(fontsize=9, loc="lower right")
plt.tight_layout()
plt.show()

## Per-Model Deep Dive

Change `SELECTED_MODEL` below to any of the six model names and re-run the cells that follow.

In [ ]:
SELECTED_MODEL = "Logistic Regression"

### Metrics for Selected Model

In [ ]:
preds = model_predictions[SELECTED_MODEL]["preds"]
probs = model_predictions[SELECTED_MODEL]["probs"]

m = evaluate(y_true, preds, probs)
for k, v in m.items():
    print(f"{k:>10s}: {v:.4f}")

### Classification Report

In [ ]:
report = classification_report(y_true, preds, output_dict=True)
pd.DataFrame(report).T

### Confusion Matrix for Selected Model

In [ ]:
cm = confusion_matrix(y_true, preds)
fig, ax = plt.subplots(figsize=(4.5, 3.5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax)
ax.set_title(f"Confusion Matrix — {SELECTED_MODEL}")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
plt.tight_layout()
plt.show()

### Prediction Preview

In [ ]:
preview = eval_df.copy()
preview["prediction"] = preds
preview.head(20)

To run the Streamlit web UI instead, use: `streamlit run app.py`